# Twitter Scrapping

## 1. Twitter Scrapping con snscrape

En esta sección se usa snscrape para hacer Twitter Scrapping de los n más recientes Tweets de la cuenta Arachno_Cosas. Esto genera un dataframe con los tweets que esta cuenta respondió a las menciones que le hicieron. Las respuestas de la cuenta consisten en la explicación de si la araña de la imagen es de importancia médica o no y la especie. 

Instalación de snscrape desde git

In [ ]:
pip install git+https://github.com/JustAnotherArchivist/snscrape.git

Seleccionar los últimos n tweeets de un usuario y guardarlos en un archivo .json:

In [482]:
import os
os.system("snscrape --jsonl --max-results 100 twitter-search 'from:Arachno_Cosas'> user-tweets.json")

0

Convertir el archivo de json a un dataframe de pandas:

In [21]:
import pandas as pd
tweets_df = pd.read_json('user-tweets.json', lines=True)
tweets_df.shape

(100, 28)

Se modifica el dataframe, quitando algunos campos que no se utilizan y filtrando únicamente los tweets que tengan mención de otro usuario, que tengan un link externo y que tengan hashtags. 

In [22]:
tweets_df = tweets_df.drop(['_type','replyCount','retweetCount','likeCount','lang','source','sourceUrl','sourceLabel',
                            'tcooutlinks','retweetedTweet','quotedTweet','inReplyToUser','coordinates',
                            'place','cashtags','quoteCount', 'media'], axis = 1)
tweets_df = tweets_df[pd.isnull(tweets_df['mentionedUsers']) == False]
tweets_df = tweets_df[pd.isnull(tweets_df['outlinks']) == False]
tweets_df = tweets_df[pd.isnull(tweets_df['hashtags']) == False]
tweets_df = tweets_df.reset_index()

In [23]:
tweets_df.shape

(10, 12)

In [24]:
tweets_df.head()

,index,url,date,content,renderedContent,id,user,conversationId,outlinks,inReplyToTweetId,mentionedUsers,hashtags
0,12,https://twitter.com/Arachno_Cosas/status/14339...,2021-09-03 23:20:13+00:00,¡Que bonito arácnido! Pertenece al orden #Soli...,¡Que bonito arácnido! Pertenece al orden #Soli...,1433932875586543616,"{'_type': 'snscrape.modules.twitter.User', 'us...",1433932875586543616,[https://twitter.com/LGLS34/status/14337957166...,NaN,"[{'_type': 'snscrape.modules.twitter.User', 'u...","[Solifugae, Eremobatidae, NIM]"
1,19,https://twitter.com/Arachno_Cosas/status/14339...,2021-09-03 21:40:46+00:00,"¡Hola, @missael0! Gracias por compartir. Perte...","¡Hola, @missael0! Gracias por compartir. Perte...",1433907848535216130,"{'_type': 'snscrape.modules.twitter.User', 'us...",1433907848535216130,[https://twitter.com/missael0/status/143383468...,NaN,"[{'_type': 'snscrape.modules.twitter.User', 'u...","[Araneidae, Neoscona, NIM]"
2,30,https://twitter.com/Arachno_Cosas/status/14338...,2021-09-03 19:09:38+00:00,"¡Hola, @_Liaga_! Gracias por compartir. Perten...","¡Hola, @_Liaga_! Gracias por compartir. Perten...",1433869813097668629,"{'_type': 'snscrape.modules.twitter.User', 'us...",1433869813097668629,[https://twitter.com/_Liaga_/status/1433849917...,NaN,"[{'_type': 'snscrape.modules.twitter.User', 'u...","[Zoropsidae, Lauricius, NIM]"
3,42,https://twitter.com/Arachno_Cosas/status/14336...,2021-09-03 02:21:22+00:00,@mitroeni ¡Hola! Se trata de una integrante de...,@mitroeni ¡Hola! Se trata de una integrante de...,1433616077053906947,"{'_type': 'snscrape.modules.twitter.User', 'us...",1433615733041270785,[https://twitter.com/Arachno_Cosas/status/1295...,1.433616e+18,"[{'_type': 'snscrape.modules.twitter.User', 'u...","[Loxosceles, IM]"
4,47,https://twitter.com/Arachno_Cosas/status/14335...,2021-09-03 01:00:13+00:00,@noviadesoquete ¡Hola! Se trata de una integra...,@noviadesoquete ¡Hola! Se trata de una integra...,1433595652722724866,"{'_type': 'snscrape.modules.twitter.User', 'us...",1433588906310701057,[https://twitter.com/Arachno_Cosas/status/1295...,1.433589e+18,"[{'_type': 'snscrape.modules.twitter.User', 'u...","[Loxosceles, IM]"


Se modifica el data frame para tomar en cuenta solo los hashtagsa que coincidan con NIM o IM. Esto se hace a través de regular expressions. Se crea una columna 'label' que indica el resultado del experto (NIM: No tiene importancia médica, IM: de importancia médica). 

In [13]:
import re

In [25]:
labels = []
patron = r'[N]?[I][M]'

for i in range(len(tweets_df)):
    texto = str(tweets_df['hashtags'][i])
    try:
        match = re.search(patron, texto)
        hashtag = match.group()
        labels.append(hashtag)
    except:
        labels.append("")

In [26]:
tweets_df['label'] = labels

In [27]:
tweets_df.label.unique()

array(['NIM', 'IM'], dtype=object)

In [28]:
tweets_df = tweets_df[tweets_df['label'] != ""]
tweets_df.label.unique()

array(['NIM', 'IM'], dtype=object)

In [29]:
tweets_df.shape

(10, 13)

In [30]:
tweets_df.head()

,index,url,date,content,renderedContent,id,user,conversationId,outlinks,inReplyToTweetId,mentionedUsers,hashtags,label
0,12,https://twitter.com/Arachno_Cosas/status/14339...,2021-09-03 23:20:13+00:00,¡Que bonito arácnido! Pertenece al orden #Soli...,¡Que bonito arácnido! Pertenece al orden #Soli...,1433932875586543616,"{'_type': 'snscrape.modules.twitter.User', 'us...",1433932875586543616,[https://twitter.com/LGLS34/status/14337957166...,NaN,"[{'_type': 'snscrape.modules.twitter.User', 'u...","[Solifugae, Eremobatidae, NIM]",NIM
1,19,https://twitter.com/Arachno_Cosas/status/14339...,2021-09-03 21:40:46+00:00,"¡Hola, @missael0! Gracias por compartir. Perte...","¡Hola, @missael0! Gracias por compartir. Perte...",1433907848535216130,"{'_type': 'snscrape.modules.twitter.User', 'us...",1433907848535216130,[https://twitter.com/missael0/status/143383468...,NaN,"[{'_type': 'snscrape.modules.twitter.User', 'u...","[Araneidae, Neoscona, NIM]",NIM
2,30,https://twitter.com/Arachno_Cosas/status/14338...,2021-09-03 19:09:38+00:00,"¡Hola, @_Liaga_! Gracias por compartir. Perten...","¡Hola, @_Liaga_! Gracias por compartir. Perten...",1433869813097668629,"{'_type': 'snscrape.modules.twitter.User', 'us...",1433869813097668629,[https://twitter.com/_Liaga_/status/1433849917...,NaN,"[{'_type': 'snscrape.modules.twitter.User', 'u...","[Zoropsidae, Lauricius, NIM]",NIM
3,42,https://twitter.com/Arachno_Cosas/status/14336...,2021-09-03 02:21:22+00:00,@mitroeni ¡Hola! Se trata de una integrante de...,@mitroeni ¡Hola! Se trata de una integrante de...,1433616077053906947,"{'_type': 'snscrape.modules.twitter.User', 'us...",1433615733041270785,[https://twitter.com/Arachno_Cosas/status/1295...,1.433616e+18,"[{'_type': 'snscrape.modules.twitter.User', 'u...","[Loxosceles, IM]",IM
4,47,https://twitter.com/Arachno_Cosas/status/14335...,2021-09-03 01:00:13+00:00,@noviadesoquete ¡Hola! Se trata de una integra...,@noviadesoquete ¡Hola! Se trata de una integra...,1433595652722724866,"{'_type': 'snscrape.modules.twitter.User', 'us...",1433588906310701057,[https://twitter.com/Arachno_Cosas/status/1295...,1.433589e+18,"[{'_type': 'snscrape.modules.twitter.User', 'u...","[Loxosceles, IM]",IM


Pasar la columna de links ('outlinks') externos a una lista:

In [59]:
links = []
for i in range(len(tweets_df)):
    link = tweets_df["outlinks"][i][0]
    links.append(link)

In [60]:
links

['https://twitter.com/LGLS34/status/1433795716690948096',
 'https://twitter.com/missael0/status/1433834683012460544',
 'https://twitter.com/_Liaga_/status/1433849917261365254',
 'https://twitter.com/Arachno_Cosas/status/1295470633669799936?s=20',
 'https://twitter.com/Arachno_Cosas/status/1295470633669799936?s=20',
 'https://twitter.com/Arachno_Cosas/status/1295470633669799936?s=20',
 'https://twitter.com/Adjloo85/status/1433458396527747073',
 'https://twitter.com/humbertoallen/status/1427441557360332805',
 'https://twitter.com/Arachno_Cosas/status/1204068593949970432?s=19',
 'https://twitter.com/Clementbm/status/1433566922491236353']

### Tweet de muestra

Ya que el data frame solo tiene un link al tweet que contiene la imagen en la que se menciona al experto es necesario hacer scrapping de esa url. 

Ejemplo:

In [32]:
tweet_index = 0

In [33]:
tweets_df.iloc[tweet_index]

index                                                              12
url                 https://twitter.com/Arachno_Cosas/status/14339...
date                                        2021-09-03 23:20:13+00:00
content             ¡Que bonito arácnido! Pertenece al orden #Soli...
renderedContent     ¡Que bonito arácnido! Pertenece al orden #Soli...
id                                                1433932875586543616
user                {'_type': 'snscrape.modules.twitter.User', 'us...
conversationId                                    1433932875586543616
outlinks            [https://twitter.com/LGLS34/status/14337957166...
inReplyToTweetId                                                  NaN
mentionedUsers      [{'_type': 'snscrape.modules.twitter.User', 'u...
hashtags                               [Solifugae, Eremobatidae, NIM]
label                                                             NIM
Name: 0, dtype: object

In [34]:
print("Url: " + str(tweets_df['url'][tweet_index]))
print("Conversation Id: " + str(tweets_df['conversationId'][tweet_index]))
print("Link externo: " + str(tweets_df['outlinks'][tweet_index]))
print("Etiqueta: " + str(tweets_df['label'][tweet_index]))
print("Contenido: " + str(tweets_df['content'][tweet_index]))

Url: https://twitter.com/Arachno_Cosas/status/1433932875586543616
Conversation Id: 1433932875586543616
Link externo: ['https://twitter.com/LGLS34/status/1433795716690948096']
Etiqueta: NIM
Contenido: ¡Que bonito arácnido! Pertenece al orden #Solifugae, de la familia #Eremobatidae probablemente. Carecen de veneno, por lo que no son considerados de importancia médica #NIM✅ Se les conoce como "arañas camello". Gracias por compartir @LGLS34. https://t.co/4xeteyNjse


In [47]:
url_externo = tweets_df['outlinks'][tweet_index][0]
url_externo

'https://twitter.com/LGLS34/status/1433795716690948096'

## 2. Tweet Scrapping con Tweepy y BeatifulSoup

En esta sección se intenta realizar la descarga de la imagen relacionada al Tweet en el que se menciona a Arachno_Cosas. Cada Tweet conteniendo la imagen está dado por la url (outlinks) del dataframe anterior. El objetivo es descargar la imagen de un Tweet a partir de su url. 

In [508]:
#!pip install tweepy

In [62]:
import tweepy
from tweepy import OAuthHandler
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin, urlparse
from requests_oauthlib import OAuth1

Autorización de Twitter, se definen las credenciales de desarrollador de Twitter (consumer_key, consumer_secret, access_token, access_secret) desde un archivo para mantenerlas secretas:

In [79]:
#pip install python-decouple
#librería para desacoplar las credenciales

In [74]:
from decouple import config

consumer_key = config('consumer_key',default='')
consumer_secret = config('consumer_secret',default='')
access_token = config('access_token',default='')
access_secret = config('access_secret',default='')

In [75]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

#### Extracción de la imagen a partir de la url

In [63]:
# url de prueba correspondiente al Tweet de muestra en la sección anterior
url = url_externo

Función para verificar que la url sea válida:

In [39]:
def is_valid(url):
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

In [49]:
is_valid(url)

True

Función para obtener todas las imágenes de la url:

In [76]:
def get_all_images(url):
    auth = OAuth1('consumer_key', 'consumer_secret',
                  'access_token', 'access_secret')
    soup = bs(requests.get(url, auth = auth).content, "html.parser")
    urls = []
    for img in tqdm(soup.find_all("img"), "Extracting images"):
        img_url = img.attrs.get("src")
        if "/thumbnail" in img_url:
            continue
        if not img_url:
            continue
        img_url = urljoin(url, img_url)
        try:
            pos = img_url.index("?")
            img_url = img_url[:pos]
        except ValueError:
            pass
        if is_valid(img_url):
            urls.append(img_url)
    return urls

In [77]:
get_all_images(url)

Extracting images: 100%|██████████| 1/1 [00:00<?, ?it/s]


['https://abs.twimg.com/errors/logo46x38.png']

Función para descargar las imágenes a un folder:

In [53]:
def download(url, pathname):
    if not os.path.isdir(pathname):
        os.makedirs(pathname)
    response = requests.get(url, stream=True)
    file_size = int(response.headers.get("Content-Length", 0))
    filename = os.path.join(pathname, url.split("/")[-1])
    progress = tqdm(response.iter_content(1024), f"Downloading {filename}", total=file_size, unit="B", unit_scale=True, unit_divisor=1024)
    with open(filename, "wb") as f:
        for data in progress:
            f.write(data)
            progress.update(len(data))

Función final:

In [54]:
def final(links, path):
    for i in links:
        imgs = get_all_images(i)
        for img in imgs:
            download(img, path)

In [61]:
final(links, "imagenes")

Extracting images: 100%|██████████| 1/1 [00:00<?, ?it/s]


#### Función alternativa para el Tweet Scrapping inicial 
(Descargar atributos de los n Tweets más recientes de una cuenta de usuario de Twitter y guardarlos a un csv). 

In [71]:
def username_tweets_to_csv(username,count):
    tweets = []
    try:      
        # creación del método de query usando parámetros
        tweets = tweepy.Cursor(api.user_timeline,id=username).items(count)

        # extraer información de tweets (objeto iterable)
        tweets_list = [[tweet.created_at, 
                        tweet.id, tweet.text, 
                        tweet.in_reply_to_status_id,
                        tweet.entities["hashtags"], 
                        tweet.in_reply_to_user_id_str, 
                        tweet.in_reply_to_screen_name] 
                       for tweet in tweets]

        # crear un data frame desde una lista tweets_list
        # añadir columnas según los atributos extraídos 
        tweets_df = pd.DataFrame(tweets_list,columns=['Datetime', 
                                                      'Tweet Id', 
                                                      'Text',
                                                      'InReplyToStatusId',
                                                      'hashtags',
                                                      'in_reply_to_user_id_strs',
                                                      'in_reply_to_screen_name'])

        # convertir el dataframe a csv
        tweets_df.to_csv('{}-tweets.csv'.format(username), sep=',', index = False)

    except BaseException as e:
        print('failed on_status,',str(e))
        time.sleep(3)